In this notebook, I will classify images from the MNIST-dataset. The datasets consist of (TODO:). The images need to be preprocessed to be normaliezd and one-hot encoded. 

## Loading Data

By importing input_data from tensorflow, you can simply call the function read_data_sets() and decide what folder to save the data, in this case, "MNIST_data/". one_hot is set to be false, because I want to show the one_hot encoding process manually. 

In [2]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"

mnist = input_data.read_data_sets("MNIST_data/", one_hot=False)


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


## Preprocessing Data

The shape of train data is (55000, 784). There are 55000 number of flattened iagme data (28 x 28 = 784). It needs to be reshaped to be 2 dimensional. We will have the input data go thorugh the tensor of shape (0, 784) and be reshaped into 28 x 28 automatically. 

In [8]:
print(mnist.train.images.shape)

(55000, 784)


In [9]:
x = tf.placeholder(tf.float32, [None, 784])
x_reshaped = tf.reshape(x, [-1, 28, 28, 1])

## Convolution layer
The following function defines creating a convolution layer including the max pool layer

In [ ]:
def create_new_conv_layer(data, num_channels, num_filters, filter_shape, pool_shape, unique_name):
    # set up the input filter shape for 2 dimensional data
    conv_filter_shape = [filter_shape[0], filter_shape[1], num_channels, num_filters]
    
    # created weights and bias for the filter
    # tf.Variable(<initial-value>, name=<optional-name>)
    
    # truncated_normal(shape, mean=0.0, stddev=1.0, dtype=dtypes.float32, seed=None, name=None) 
    # outputs random values from a truncated normal distribution.
    weights = tf.Variable(tf.truncated_normal(conv_filter_shape, stddev=0.03), name='w_' + unique_name)
    bias = tf.Variable(tf.truncated_normal([nu_filters]), name='b_' + unique_name)
    
    # set up the convolutional layer operation
    # Computes a 2-D convolution given 4-D input and filter tensors.
    # Given an input tensor of shape [batch, in_height, in_width, in_channels] and a filter / kernel tensor of shape 
    # [filter_height, filter_width, in_channels, out_channels], this op performs the following:
    # 1. Flattens the filter to a 2-D matrix with shape [filter_height * filter_width * in_channels, output_channels].
    # 2. Extracts image patches from the input tensor to form a virtual tensor of shape 
    # [batch, out_height, out_width, filter_height * filter_width * in_channels].
    # 3. For each patch, right-multiplies the filter matrix and the image patch vector.
    conv_out_layer = tf.nn.conv2d(data, weights, [1, 1, 1, 1], padding='SAME')
    
    # add the bias to the output of the convolutional layer
    conv_out_layer += bias
    
    # apply a non-linear activation (ReLU)
    conv_out_layer = tf.nn.relu(conv_out_layer)
    
    # apply max pooling
    # ksize is the argument which defines the size of the max pooling window (i.e. the area over which the maximum is
    # calculated).  It must be 4D to match the convolution - in this case, for each image we want to use a 2 x 2 area
    # applied to each channel
    ksize = [1, pool_shape[0], pool_shape[1], 1]
    
    # strides defines how the max pooling area moves through the image - a stride of 2 in the x direction will lead to
    # max pooling areas starting at x=0, x=2, x=4 etc. through your image.  If the stride is 1, we will get max pooling
    # overlapping previous max pooling areas (and no reduction in the number of parameters).  In this case, we want
    # to do strides of 2 in the x and y directions.
    strides = [1, 2, 2, 1]
    
    out_layer = tf.nn.max_pool(conv_out_layer, ksize=ksize, strides=strides, padding='SAME')
    
    return out_layer